In [12]:
import os
import requests
import zipfile
import shutil
import numpy as np
from tqdm import tqdm

In [13]:
# for downloading all KML files (need to update url list manually)
def download_kmls(download_dir_kmls_zip, kml_url_list=None, download_zip=True):
    if not kml_url_list:
        kml_url_list = ['https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle1_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle2_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle3_date_time_0.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle4_date_time_rev2_0.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle5_date_time_rev3.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle6_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle7_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle8_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle9_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle10_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle11_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2RGTscycle12datetime.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle13_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle14_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle15_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle16_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle17_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle18_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle19_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle20_date_time.zip',
                        'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle21_date_time_provisional.zip'
                       ]

    # Download the Zip files
    if not os.path.exists(download_dir_kmls_zip):
        os.makedirs(download_dir_kmls_zip)
        
    if download_zip: 
        for url in kml_url_list:
            kml_zip_fn = url[url.rfind('/')+1:]
            kml_zip_path = download_dir_kmls_zip + '/' + kml_zip_fn
            print('downloading', kml_zip_path)
            response = requests.get(url, stream=True, allow_redirects=True)
            total_size_in_bytes= int(response.headers.get('content-length', 0))
            block_size = 1024 #1 Kibibyte
            progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
            with open(kml_zip_path, 'wb') as file:
                for data in response.iter_content(block_size):
                    progress_bar.update(len(data))
                    file.write(data)
            progress_bar.close()
            if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
                print("ERROR, something went wrong")

    # Unzip outputs
    for z in os.listdir(download_dir_kmls_zip): 
        if z.endswith('.zip'): 
            zip_name = download_dir_kmls_zip + "/" + z 
            print('--> extracting', zip_name)
            zip_ref = zipfile.ZipFile(zip_name) 
            zip_ref.extractall(download_dir_kmls_zip) 
            zip_ref.close() 
            os.remove(zip_name)

    # Clean up Outputs folder by removing individual granule folders 
    print('Cleaning up outputs folder...', end=' ')
    for root, dirs, files in os.walk(download_dir_kmls_zip, topdown=False):
        for file in files:
            try:
                shutil.move(os.path.join(root, file), download_dir_kmls_zip)
            except OSError:
                pass
        for name in dirs:
            shutil.rmtree(root+'/'+name, ignore_errors=True)
    print(' --> DONE!')

    kml_filelist = [download_dir_kmls_zip+'/'+f for f in os.listdir(download_dir_kmls_zip) \
                    if os.path.isfile(os.path.join(download_dir_kmls_zip, f))]
    for i in np.arange(1,21):
        cycle_search = '_cycle%i_' % i
        if i == 1: 
            cycle_filelist = [f for f in kml_filelist if (('_cycle' not in f) & (f[len(download_dir_kmls_zip)+1] != '.'))]
        else:
            cycle_filelist = [f for f in kml_filelist if ((cycle_search in f) & (f[len(download_dir_kmls_zip)+1] != '.'))]
        print('cycle %2i: %4i KMLs' % (i, len(cycle_filelist)))

In [ ]:
# download_path = '/home/jovyan/shared-public/ICESat-2-Hackweek/ground_tracks'
download_path = '/home/jovyan/shared-public/ICESat-2-Hackweek/KMLs'

url_list = ['https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle16_date_time.zip',
            'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle17_date_time.zip',
            'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle18_date_time.zip',
            'https://icesat-2.gsfc.nasa.gov/sites/default/files/page_files/IS2_RGTs_cycle19_date_time.zip'
            ]

download_kmls(download_path, kml_url_list=url_list, download_zip=True)

downloading /home/jovyan/shared-public/ICESat-2-Hackweek/KMLs/IS2_RGTs_cycle16_date_time.zip


100%|██████████| 148M/148M [44:56<00:00, 54.9kiB/s]   


downloading /home/jovyan/shared-public/ICESat-2-Hackweek/KMLs/IS2_RGTs_cycle17_date_time.zip


100%|██████████| 148M/148M [46:28<00:00, 53.1kiB/s]   


downloading /home/jovyan/shared-public/ICESat-2-Hackweek/KMLs/IS2_RGTs_cycle18_date_time.zip


 11%|█▏        | 17.0M/148M [05:25<37:14, 58.6kiB/s]  